In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from selenium import webdriver  
from selenium.webdriver.common.action_chains import ActionChains  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4.element import NavigableString
import time
import re
import requests
import os
import pandas as pd 
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import gc
gc.collect()

0

In [3]:
os.chdir('C:/Users/USER/Desktop/Text Mining_YHK/0_NewsCrawer')
print(os.getcwd())

C:\Users\USER\Desktop\Text Mining_YHK\0_NewsCrawer


### **1. IPO Data**
* Create IPO dataframe based on each season
* IPO Data processing : 
    * create date range for news searching
    * drop foreign company(KY)

In [5]:
IPO = pd.read_excel('IPO_RawData.xlsx')
IPO = IPO[IPO['ipo_date'] >= '2019-01-01'].reset_index(drop=True)
#print(IPO)

In [6]:
def trans_dtime(df):
    df['ipo_date'] = pd.to_datetime(df['ipo_date'])
    df['establish_date'] = pd.to_datetime(df['establish_date'])

trans_dtime(IPO)

In [7]:
print(type(IPO.loc[0:1,'ipo_date']))

<class 'pandas.core.series.Series'>


In [8]:
def clean(df):
    df['stock_code'] = df['name'].str[:4]
    df['name'] = df['name'].str[5:]
    df['age'] = df['ipo_date'] - IPO['establish_date']
    df['age'] = df['age']/timedelta(days=365)
    df['ipo_year'] = df['ipo_date'].dt.year
    df['allot'] = df['allot']*0.01 #平均中籤率
    df['ipo_month'] = df['ipo_date'].dt.month

clean(IPO)
IPO['KY'] = 0
IPO.loc[IPO['name'].str.contains('KY'),'KY'] = 1 #名字有KY的 KY欄輸1

IPO['underprice'] = IPO['close_price']/IPO['offer_price'] - 1 ##有missing '-' 

IPO['oversub'] = 1/IPO['allot'] 

# create news search range
#   為了包含某公司的前一個月到IPO前一天的新聞
s = [] 
e = []
    
for j in IPO['ipo_date']:
    s.append((j - relativedelta(months=1)).strftime("%Y-%m-%d"))
    
for i in IPO['ipo_date']:
    e.append((i- relativedelta(days=1)).strftime("%Y-%m-%d"))

IPO['start_date'] = s
IPO['end_date'] = e

# USE_CODE = IPO[IPO['KY'] == 0][['stock_code']].sort_values(by='stock_code', ascending = 'False').reset_index(drop = True)

##處裡過的IPO df
IPO_USE = IPO[IPO['KY'] == 0].reset_index(drop = True) 

In [9]:
def ipo_group(i,j):
    return IPO_USE[(IPO_USE['ipo_year'] == i) & IPO_USE['ipo_month'].isin(j)].reset_index(drop = True)

IPO_1904, IPO_1908, IPO_1912 = ipo_group(2019,list(range(1,5))), ipo_group(2019,list(range(5,9))), ipo_group(2019,list(range(9,13)))
IPO_2004, IPO_2008, IPO_2012 = ipo_group(2020,list(range(1,5))), ipo_group(2020,list(range(5,9))), ipo_group(2020,list(range(9,13)))
IPO_2104, IPO_2108, IPO_2112 = ipo_group(2021,list(range(1,5))), ipo_group(2021,list(range(5,9))), ipo_group(2021,list(range(9,13)))
IPO_2204, IPO_2208, IPO_2212 = ipo_group(2022,list(range(1,5))), ipo_group(2022,list(range(5,9))), ipo_group(2022,list(range(9,13)))
IPO_2304, IPO_2308, IPO_2312 = ipo_group(2023,list(range(1,5))), ipo_group(2023,list(range(5,9))), ipo_group(2023,list(range(9,13)))
IPO_2404, IPO_2408, IPO_2412 = ipo_group(2024,list(range(1,5))), ipo_group(2024,list(range(5,9))), ipo_group(2011,list(range(9,13)))

In [10]:
IPO_1904[['name','ipo_year','ipo_month']]

,name,ipo_year,ipo_month
0,十銓,2019,1
1,大詠城,2019,1
2,錸寶,2019,1
3,易發,2019,1
4,M31,2019,1
5,詠昇,2019,2
6,元翎,2019,3
7,勤凱,2019,3
8,樂斯科,2019,3
9,緯穎,2019,3


In [11]:
print('年:',IPO_1904['ipo_year'].unique(),' 月：',IPO_1904['ipo_month'].unique())

年: [2019]  月： [1 2 3 4]


In [12]:
IPO[IPO['name'] == '易發'][['name','start_date','end_date']]

,name,start_date,end_date
3,易發,2018-12-18,2019-01-17


### **2. WebCrawler Function**
* open news page
* collect news title date name for each company
* collect news content for each company
* save as excel file

In [14]:
def open_udn():
    driver.implicitly_wait(10)
    driver.maximize_window()
    driver.get('https://udndata.com/ndapp/Index?cp=udn')
    # fixed ip log in
    #driver.find_element("xpath", "/html/body/div[@id='container']/main[@class='index']"\
    #                             +"/div[@class='wrapper clearfix']/aside[@class='service']"\
    #                             +"/div[@class='member']/div[@class='sign-in']").click()
    driver.find_element("xpath", "/html/body/div[@id='container']/header/"\
                                    +"div[@class='wrapper top clearfix']/"\
                                    +"ul[@class='menu clearfix']/li[2]/a[@class='fa-group']").click()

    time.sleep(1)

In [15]:
# get news_title, date, name; use in get_news() 
def get_title_date_name(x,y):
    # title, append to title list
    for title in x.find_all(class_ = "control-pic"):
        title_list.append(title.a.string.replace("\n",""))
    # date, append to date list
    for date in x.find_all(class_ = "source"):
        date_list.append(date.string[:10])
    # name, append to name list
    for i in range(len(x.find_all(class_ = "control-pic"))):
        name_list.append(y)

In [16]:
# get news content; use in get_news()
def get_content(j):
    #open new news tab 
    new_tab = driver.find_element("xpath", "/html/body/div/main/div/div[1]/section/div[6]/ul/li["+str(j)+"]/div/h2/a") 
    action = ActionChains(driver)
    action.move_to_element(new_tab).key_down(Keys.CONTROL).click(new_tab).key_up(Keys.CONTROL).perform() # new tab
    time.sleep(3) # wait for new tab 
    driver.switch_to.window(driver.window_handles[1]) # switch to new tab
    time.sleep(1)
    
    # check log in status
    log_in = BeautifulSoup(driver.page_source,"lxml")
    if log_in.find(string = "會員登入") == "會員登入":
        # login again
        driver.find_element("xpath","/html/body/div/main/div/div[1]/section/div[3]/a[1]").click()
    elif log_in.find(string = "錯誤訊息") == "錯誤訊息":
        # refresh window
        driver.refresh()
        time.sleep(1)
    else:
        pass
    # get content of new tab, append to content list
    bsobj = BeautifulSoup(driver.page_source,"lxml") 
    content = bsobj.find("article")
    content_list.append(content.get_text().replace("\n","")\
                        .replace("\t","").replace("\u3000","").replace(" ",""))
    time.sleep(2)
    
    # close new tab
    driver.close()
    time.sleep(1)
    
    # switch back to search result page
    driver.switch_to.window(driver.window_handles[0])
    

In [17]:
def get_news(name,start_date,end_date):
    # change url
    url = driver.current_url
    driver.get(url)

    # input search element
    search_elem = driver.find_element("xpath",'/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/div[1]/div[1]/input')
    search_elem.send_keys(name) # input company name
    time.sleep(1)
    
    # input search date range
    start = driver.find_element("id",'datepicker-start')
    start.send_keys(u""+start_date)
    time.sleep(1)
    end = driver.find_element("id",'datepicker-end')
    end.send_keys(u""+end_date)
    time.sleep(1)
    
    # click on search button
    driver.find_element("xpath","/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/button").click()

    # click on show 50 results in one page button
    select = Select(driver.find_element("id",'sharepage'))
    select.select_by_value("50")
    time.sleep(1)

    # Find news result number
    news = BeautifulSoup(driver.page_source, "lxml")
    time.sleep(1)
    news_num = int(news.find_all(class_= "mark")[1].text)

    # Find news page
    if (news_num/50).is_integer():
        news_page = int(news_num/50) 
    else:
        news_page = int(news_num/50)+1

    ## For loop to change page and get news content
    for i in range(0,news_page+1):
        # no page
        if i == 0:
            pass
        # page 1
        elif i == 1:
            get_title_date_name(news,name)
            for j in range(1,(news_num-(i*50-51))):
                if j <= 50:
                    get_content(j)
        # other page
        else:
            #click to change page
            driver.find_element("xpath","/html/body/div[@id='container']/main/div[@class='wrapper clearfix']/div[@id='mainbar']"\
                                         +"/section[@class='list-news']/div[@class='page-number page-number-web']/a["+str(i+2)+"]").click()
            driver.get(driver.current_url)
            news = BeautifulSoup(driver.page_source, "lxml")
            get_title_date_name(news,name)
            time.sleep(1)
            for j in range(1,(news_num-(i*50-51))):
                if j <= 50:
                    get_content(j)
    
    ### return to search page
    driver.find_element("xpath","/html/body/div/header/div[2]/div/div[1]/div/a[2]").click()
    gc.collect()
    time.sleep(1)

In [53]:
def excel(title):
    df = pd.DataFrame({'name':name_list,'date':date_list,'title':title_list,'content':content_list})
    df.to_excel('./IPO_Data/'+title+'.xlsx', index=False)
    return df

###  **3. examination**

In [144]:
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

In [146]:
# change url
url = driver.current_url
driver.get(url)

# input search element
search_elem = driver.find_element("xpath",'/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/div[1]/div[1]/input')
search_elem.send_keys("碩天") # input company name
time.sleep(1)
    
# input search date range
start = driver.find_element("id",'datepicker-start')
start.send_keys(u"2009-11-23")
time.sleep(1)
end = driver.find_element("id",'datepicker-end')
end.send_keys(u"2009-12-22")
time.sleep(1)
    
# click on search button
driver.find_element("xpath","/html/body/div[1]/main/div/div[1]/section[1]/div[2]/form/button").click()

In [148]:
# click on show 50 results in one page button
select = Select(driver.find_element("id",'sharepage'))
select.select_by_value("50")
time.sleep(1)

# Find news result number
news = BeautifulSoup(driver.page_source, "lxml")
time.sleep(1)
news_num = int(news.find_all(class_= "mark")[1].text)

# Find news page
if (news_num/50).is_integer():
    news_page = int(news_num/50) 
else:
    news_page = int(news_num/50)+1

In [150]:
title_list = []
date_list = []

In [152]:
for title in news.find_all(class_ = "control-pic"):
    title_list.append(title.a.string.replace("\n",""))
# date, append to date list
for date in news.find_all(class_ = "source"):
    date_list.append(date.string[:10])

In [154]:
print(title_list[:5])
print(date_list[:5])

['1.碩天科技23日 掛牌上市', '2.五公司 趕搭資金行情及時車', '3.看ECFA發酵程度', '4.蜜月新人 今年台股包大禮', '5.新股領軍 元月行情可期']
['2009-12-21', '2009-12-21', '2009-12-20', '2009-12-19', '2009-12-19']


In [156]:
new_tab = driver.find_element("xpath", "/html/body/div/main/div/div[1]/section/div[6]/ul/li["+str(1)+"]/div/h2/a") 
action = ActionChains(driver)
action.move_to_element(new_tab).key_down(Keys.CONTROL).click(new_tab).key_up(Keys.CONTROL).perform() # new tab
time.sleep(3) # wait for new tab 
driver.switch_to.window(driver.window_handles[1]) # switch to new tab
time.sleep(1)

bsobj = BeautifulSoup(driver.page_source,"lxml") 
content_list = []
content = bsobj.find("article")
#print(type(content))

content_list.append(content.get_text().replace("\n","")\
                        .replace("\t","").replace("\u3000","").replace(" ",""))

In [158]:
#print(bsobj.prettify())
print(content_list[:5])
print(type(content))
#/html/body/div/main/div/div[1]/section/div[6]/ul/li["+str(j)+"]/div/h2/a

['以製造不斷電電源系統(UPS)揚名海外的碩天科技公司在日前已舉辦上市前法說會，並預計於23日正式掛牌。碩天科技於1997年成立，產品包括不斷電電源系統(UPS)、可攜式電源、電源保護及電源管理等，並藉由自有品牌(CyberPower)銷售至歐美地區。客戶包括美國主要零售商BestBuy、CircuitCity和Wal-Mart及經銷商IngramMicro、TechData和D&H等。碩天科技目前專注於生產小功率(5KVA以下)的UPS產品，並主打美國市場，目前已成為美國零售通路前兩大品牌廠商。碩天總經理何溓洵表示，因應公司發展，碩天科技今年開始增加中功率(5KVA~10KVA)產品線與新通路據點，包括日本、奈及利亞等地區，將持續開發經銷商通路市場，預期未來幾年將持續帶動公司成長。(項孟汝)']
<class 'bs4.element.Tag'>


In [ ]:
for title in news.find_all(class_ = "control-pic"):
    print(title.a.string.replace("\n",""))
    link = "https://udndata.com" + title.a["href"]
#     print(link)
    resp = requests.get(link)
    soup = BeautifulSoup(resp.text, 'lxml')
#     content = soup.find("article")
    #print(soup)

In [ ]:
#print([x for x in name_list if x =='易發'])
print(IPO_1904['name'])
#print(date_list[:5])
#print(content_list[:5])

### **4. Run by every four months**

* 2019

In [186]:
## 2019/01~04
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()


title_list = []
date_list = []
name_list = []
content_list = []

for i,j,k in zip(IPO_1904[~IPO_1904['name'].isin(['易發'])]['name'], IPO_1904[~IPO_1904['name'].isin(['易發'])]['start_date'], IPO_1904[~IPO_1904['name'].isin(['易發'])]['end_date']):
    get_news(i,j,k)



driver.quit()
df_1904 = excel('1904')

In [188]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2019/05~08
for i,j,k in zip(IPO_1908['name'],IPO_1908['start_date'], IPO_1908['end_date']):
    get_news(i,j,k)

driver.quit()
df_1908 = excel('1908')

In [189]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2019/09~12
for i,j,k in zip(IPO_1912['name'],IPO_1912['start_date'], IPO_1912['end_date']):
    get_news(i,j,k)

driver.quit()
df_1912 = excel('1912')

* 2020

In [ ]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2020/01~04
for i,j,k in zip(IPO_2004['name'],IPO_2004['start_date'], IPO_2004['end_date']):
    get_news(i,j,k)

driver.quit()
df_2004 = excel('2004')

In [57]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2020/05~08
for i,j,k in zip(IPO_2008['name'],IPO_2008['start_date'], IPO_2008['end_date']):
    get_news(i,j,k)

driver.quit()
df_2008 = excel('2008')

In [58]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2020/09~12
for i,j,k in zip(IPO_2012['name'],IPO_2012['start_date'], IPO_2012['end_date']):
    get_news(i,j,k)

driver.quit()
df_2012 = excel('2012')

* 2021

In [63]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2021/01~04
for i,j,k in zip(IPO_2104['name'],IPO_2104['start_date'], IPO_2104['end_date']):
    get_news(i,j,k)

driver.quit()
df_2104 = excel('2104')

In [64]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2121/05~08
for i,j,k in zip(IPO_2108['name'],IPO_2108['start_date'], IPO_2108['end_date']):
    get_news(i,j,k)

driver.quit()
df_2108 = excel('2108')

In [65]:
# drive chrome
cService = webdriver.ChromeService(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(service = cService)
open_udn()

title_list = []
date_list = []
name_list = []
content_list = []

# get news of IPOs in 2121/09~12
for i,j,k in zip(IPO_2112['name'],IPO_2112['start_date'], IPO_2112['end_date']):
    get_news(i,j,k)

driver.quit()
df_2112 = excel('2112')